In [3]:
# Install ultralytics and opencv now that torch is ready
import sys, subprocess
def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
pip_install("ultralytics>=8.2.0")
pip_install("opencv-python")
print("Installed ultralytics and opencv-python.")

Installed ultralytics and opencv-python.


In [ ]:
# Reinstall a stable CPU-only PyTorch version compatible with Ultralytics
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "--index-url", "https://download.pytorch.org/whl/cpu", "torch==2.4.0+cpu", "torchvision==0.19.0+cpu", "torchaudio==2.4.0+cpu"])
print("Installed pinned torch CPU builds (2.4.0).")

Installed torch CPU builds.


In [4]:
# YOLOv11n video inference
import os
from ultralytics import YOLO
import cv2
from datetime import datetime

# Configure paths
video_folder = r"c:\Users\Rishabh Thapliyal\Desktop\Springboard"  # change if needed
# Pick the first video file found (mp4, avi, mov, mkv)
video_exts = (".mp4", ".avi", ".mov", ".mkv")
video_path = None
for name in os.listdir(video_folder):
    if name.lower().endswith(video_exts):
        video_path = os.path.join(video_folder, name)
        break
assert video_path is not None, "No video file found in the folder. Please place a video and rerun."
print(f"Using video: {video_path}")

# Output file
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = os.path.join(video_folder, f"yolo11n_output_{ts}.mp4")
print(f"Saving annotated video to: {out_path}")

# Load YOLOv11n model
model = YOLO("yolo11n.pt")  # downloads weights if missing

# Read input video
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), f"Failed to open video: {video_path}"
fps = cap.get(cv2.CAP_PROP_FPS) or 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

frame_idx = 0
try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        # Run prediction on the current frame
        results = model.predict(source=frame, imgsz=max(640, max(width, height)), conf=0.25, verbose=False)
        # results is a list; take first item
        r = results[0]
        # Plot annotations on the frame
        annotated = r.plot()
        writer.write(annotated)
        frame_idx += 1
        if frame_idx % 50 == 0:
            print(f"Processed {frame_idx} frames...")
finally:
    cap.release()
    writer.release()
print(f"Done. Output saved to: {out_path}")

# Show first frame of output for quick visual check
cap2 = cv2.VideoCapture(out_path)
ok, f0 = cap2.read()
cap2.release()
if ok:
    from matplotlib import pyplot as plt
    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(f0, cv2.COLOR_BGR2RGB))
    plt.title("First annotated frame")
    plt.axis("off")
else:
    print("Could not read back from output video.")

AttributeError: partially initialized module 'torch' has no attribute 'version' (most likely due to a circular import)